In [1]:
import numpy as np
import scipy as sp
import helper as h
from functools import partial
import matplotlib.pyplot as plt
import pandas as pd
import os

import torch
import torch.nn as nn


In [2]:
%reload_ext autoreload
%autoreload 2

# running pipeline

In [ ]:
n, p = 1000, 1000
r, s = 1, 1

np.random.seed(0)
X_train = h.gaussian_sample(n, p)[0]
beta = h.beta_sampler(p, r = np.sqrt(n) * r)
# have to rescale to keep distribution the same
X_test = np.random.normal(size = (1000, p)) / np.sqrt(n)
y_test = X_test @ beta
lambdas = np.logspace(-1, 2, 30)

results_gaussian = h.gcv_test_pipeline(
    X_train = X_train,
    y_train = None,
    X_test = X_test,
    y_test = y_test,
    lambdas = lambdas,
    r = r,
    sigma = s,
    k = 0,
    true_beta = beta,
    n_iters = 10
)

In [ ]:
# now write a plotter function
# need 5 subplots: actual MSE curves, gcv MSE curves, LOOCV mse curves, new GCV MSE curves, and finally all 4 in 1 plot for 1 noise iteration

h.plot_results(results_gaussian, lambdas, 1, filename = "neurips/gaussian-square.png")
# h.plot_results(results_gaussian, lambdas, 1, )

In [ ]:
h.plot_results(results_gaussian, lambdas, 1, filename = "neurips/gaussian.png", square=False)

In [ ]:
n, p = 1000, 1000
r, s = 2, 1

np.random.seed(0)
X_train = h.gaussian_sample(n, p)[0]
beta = h.beta_sampler(p, r = np.sqrt(n) * r)
# have to rescale to keep distribution the same
X_test = np.random.normal(size = (1000, p)) / np.sqrt(n)
y_test = X_test @ beta
lambdas = np.logspace(-2, 1, 30)

results = h.gcv_test_pipeline(
    X_train = X_train,
    y_train = None,
    X_test = X_test,
    y_test = y_test,
    lambdas = lambdas,
    r = r,
    sigma = s,
    k = 0,
    true_beta = beta,
    n_iters = 10
)

In [ ]:
h.plot_results(results, lambdas, 1, None)

### autocorrelated example

In [ ]:
n = 1000
p = 1000
r = 1
s = 1
np.random.seed(0)
X_train = h.rho_auto_sample(n, p, rho = 0.8)
beta = h.beta_sampler(p, r = r * np.sqrt(n))
X_test = h.rho_auto_sample(n, p, rho = 0.8)
y_test = X_test @ beta
lambdas = np.logspace(-1, 2, 30)

results_auto = h.gcv_test_pipeline(
    X_train = X_train,
    y_train = None,
    X_test = X_test,
    y_test = y_test,
    lambdas = lambdas,
    r = r,
    sigma = s,
    k = 0,
    true_beta = beta,
    n_iters = 10
)

In [ ]:
h.plot_results(results_auto, lambdas, 1, "neurips/auto.png", square = False)

## t distributed

In [ ]:
n = 1000
p = 1000
r = 1
s = 1
np.random.seed(0)
X_train = h.t_sample(n, p)
beta = h.beta_sampler(p, r = r * np.sqrt(n))
X_test = h.t_sample(n, p)
y_test = X_test @ beta
lambdas = np.logspace(-1, 2, 30)

results_t = h.gcv_test_pipeline(
    X_train = X_train,
    y_train = None,
    X_test = X_test,
    y_test = y_test,
    lambdas = lambdas,
    r = r,
    sigma = s,
    k = 0,
    true_beta = beta,
    n_iters = 10
)

In [ ]:
h.plot_results(results_t, lambdas, 1, "neurips/t.png", square=False,)

### equicorrelated

In [ ]:
n = 1000
p = 1000
r = 1
s = 1
np.random.seed(0)
X_train = h.latent_base_sample(n, p, rho = 0.8)
beta = h.beta_sampler(p, r = r * np.sqrt(n))
X_test = h.latent_base_sample(n, p, rho = 0.8) # h.gaussian_sample(n, p)[0]
y_test = X_test @ beta
lambdas = np.logspace(-1, 2, 30)

results_equicorrelated = h.gcv_test_pipeline(
    X_train = X_train,
    y_train = None,
    X_test = X_test,
    y_test = y_test,
    lambdas = lambdas,
    r = r,
    sigma = s,
    k = 0,
    true_beta = beta,
    n_iters = 10
)

In [ ]:
h.plot_results(results_equicorrelated, lambdas, 1, "neurips/equi.png", ylim = [0, 2.0], square=False,)


# additional experiments

## sp500 experiments

In [84]:
import pickle

with open("gcv_train_30.pkl", "rb") as f:
    gcv_train = pickle.load(f).to_numpy()

with open("gcv_test_30.pkl", "rb") as f:
    gcv_test = pickle.load(f).to_numpy()

In [ ]:
gcv_train.shape

In [ ]:
gcv_test.shape

In [ ]:
n, p = 493, 493
r, s = 1, 1

X_train = gcv_train # h.gaussian_sample(n, p)[0]
results_array = [None for _ in range(4)]

for i in range(4):
    np.random.seed(i)
    beta = h.beta_sampler(p, r = np.sqrt(n) * r)
    # have to rescale to keep distribution the same
    X_test = gcv_test # np.random.normal(size = (1000, p)) / np.sqrt(n)
    y_test = X_test @ beta
    lambdas = np.logspace(-1, 2, 30)

    results_array[i] = h.gcv_test_pipeline(
        X_train = X_train,
        y_train = None,
        X_test = X_test,
        y_test = y_test,
        lambdas = lambdas,
        r = r,
        sigma = s,
        k = 0,
        true_beta = beta,
        n_iters = 10,
        oracle_test=False
    )

In [ ]:
h.plot_results_combo(results_array, lambdas, 1, filename = "neurips/sp500_ensemble.png")

In [ ]:
h.plot_results_combo(results_array[2:4], lambdas, 1, filename = "neurips/sp500_ensemble_2of2.png")

In [ ]:
h.plot_results(results_array[0], lambdas, 1, "neurips/sp500_single.png", square=False,)

# example with alignment

In [75]:
n, p = 1000, 1000
k = 10

np.random.seed(0)

X, Qt, d, O = h.rho_auto_sample(n, p, rho = 0.8, return_svd=True) # already scaled down
# print(O[0])
D = np.zeros((n, p))
np.fill_diagonal(D, d)
# D[0, 0] = 40

# X = Qt @ D @ O

X_tmp, Qt, d, O_unused = h.rho_auto_sample(n, p, rho = 0.8, return_svd = True) # this way don't need to rescale
O_new = h.conditional_haar_draw(O, k = k)
D_new = np.zeros((n, p))
np.fill_diagonal(D_new, d)
# D_new[0, 0] = 40

X_test = Qt @ D_new @ O_new # X_tmp @ O_unused.T @ O_new # cancels out old and gets new one in

beta_unaligned = h.beta_sampler(p, r = 1 * np.sqrt(n))
alphas = (np.sqrt(n) * (np.arange(k) + 1) / k)[::-1]
beta_aligned = np.zeros(p)
for i in range(k):
    beta_aligned += alphas[i] * O[i]

beta = beta_unaligned + beta_aligned

In [ ]:
np.allclose(O_new[:k], O[:k])

In [ ]:
alphas

In [ ]:
lambdas = np.logspace(-1, 1, 30)
result_explicit_align = h.gcv_test_pipeline(
    X_train = X,
    y_train = None,
    X_test = X_test,
    true_beta=beta,
    y_test = X_test @ beta,
    lambdas = lambdas,
    r = np.linalg.norm(beta) / np.sqrt(n),
    sigma = 1,
    k = 10,
    n_iters = 10
)

In [ ]:
h.plot_results(result_explicit_align, lambdas, 1, "neurips/aligned-square.png", ylim = [0, 2.5])

In [ ]:
h.plot_results(result_explicit_align, lambdas, 1, "neurips/aligned.png", ylim = [0, 2.5], square=False)

In [ ]:
lambdas = np.logspace(-1, 1, 30)
result_explicit_align_nok = h.gcv_test_pipeline(
    X_train = X,
    y_train = None,
    X_test = X_test,
    true_beta=beta,
    y_test = X_test @ beta,
    lambdas = lambdas,
    r = np.linalg.norm(beta) / np.sqrt(n),
    sigma = 1,
    k = 0,
    n_iters = 10
)

In [ ]:
h.plot_results(result_explicit_align_nok, lambdas, 1, "neurips/aligned_bad.png", ylim = [0, 35.5])

# gaussian mixture

In [ ]:
n, p = 1000, 1000
r, s = 1, 1

np.random.seed(0)

X_train = h.gaussian_mixture_sample(n, p, mu = 3)[0]
beta = h.beta_sampler(p, r = np.sqrt(n) * r)
# have to rescale to keep distribution the same
X_test = h.gaussian_mixture_sample(n, p, mu = 3)[0]
y_test = X_test @ beta
lambdas = np.logspace(-1, 1, 30)

results = h.gcv_test_pipeline(
    X_train = X_train,
    y_train = None,
    X_test = X_test,
    y_test = y_test,
    lambdas = lambdas,
    r = r,
    sigma = s,
    k = 10,
    true_beta = beta,
    n_iters = 10,
    oracle_test=False
)

In [ ]:
h.plot_results(results, lambdas, 1, "neurips/mixture.png", square=False)
# h.plot_results(results, lambdas, 1, None)

In [ ]:
n, p = 1000, 1000
r, s = 1, 1

np.random.seed(0)

X_train = h.gaussian_row_corr_sample(n, p, rho = 0.6)[0]
beta = h.beta_sampler(p, r = np.sqrt(n) * r)
# have to rescale to keep distribution the same
X_test = h.gaussian_row_corr_sample(n, p, rho = 0.6)[0]
y_test = X_test @ beta
lambdas = np.logspace(-1, 1, 30)

results_row_equi = h.gcv_test_pipeline(
    X_train = X_train,
    y_train = None,
    X_test = X_test,
    y_test = y_test,
    lambdas = lambdas,
    r = r,
    sigma = s,
    k = 10,
    true_beta = beta,
    n_iters = 10,
    oracle_test=False
)

In [ ]:
h.plot_results(results_row_equi, lambdas, 1, "neurips/row_equicorrelation.png", square=False)

In [ ]:
n, p = 1000, 1000
r, s = 1, 1

X_train = h.gaussian_mixture_sample(n, p, mu = 3)[0]
beta = h.beta_sampler(p, r = np.sqrt(n) * r) + np.ones(p) / np.sqrt(p) * 2
# have to rescale to keep distribution the same
X_test = h.gaussian_mixture_sample(n, p, mu = 3)[0]
y_test = X_test @ beta
lambdas = np.logspace(-1, 1, 30)

results_mixture_spike = h.gcv_test_pipeline(
    X_train = X_train,
    y_train = None,
    X_test = X_test,
    y_test = y_test,
    lambdas = lambdas,
    r = r,
    sigma = s,
    k = 1,
    true_beta = beta,
    n_iters = 10
)

In [ ]:
h.plot_results(results_mixture_spike, lambdas, 1, "neurips/mixture_spike.png")